In [81]:
# problem set up: 
import numpy as np
states = ['1','2','3','4','5'.'6','7','8','9','10','11','12','13','14']
actions = ['up', 'down', 'left', 'right']
rewards = [[-1, -1, 0, -1], 
           [-1, -1, -1, -1], 
           [-1, -1, -1, -1], 
           [0, -1, -1, -1],  
           [-1, -1, -1, -1], 
           [-1, -1, -1, -1], 
           [-1, -1, -1, -1],  
           [-1, -1, -1, -1], 
           [-1, -1, -1, -1],  
           [-1, -1, -1, -1], 
           [-1, 0, -1, -1], 
           [-1, -1, -1, -1], 
           [-1, -1, -1, -1], 
           [-1, -1, -1, 0], ]

# states, possibilities = transition[curr_state][action]
# states are numbered by the index of "states" list defined above.
# for each state (row), the probability of executing each action (column)
transition = [[([1],[1]), ([2],[1]), ([3],[1])], [([7],[1]), ([4],[1]), ()], 
              [([4, 7],[0.5, 0.5]), ([4],[1]), ([5],[1])],
              [([5, 8],[0.5, 0.5]), ([5],[1]), ()], 
              [([6],[1]), ([7],[1]), ([8],[1])], 
              [([9],[1]), ([8],[1]), ()],
              [([10],[1]), ([10],[1]),([10],[1]),([10],[1])], 
              [([10],[1]), ([10],[1]),([10],[1]),([10],[1])],
              [([10],[1]), ([10],[1]),([10],[1]),([10],[1])], 
              [([10],[1]), ([10],[1]),([10],[1]),([10],[1])]]
startState = 0
endState = 10


In [88]:
# helper functions
def get_actions(state):
    act = [a for a in range(len(actions)) if not np.isnan(rewards[state][a])]
    return act

def is_end(state):
    return state == endState

def get_next_state(curr_state,action):
    states, poss = transition[curr_state][action]
    if len(states) == 1:
        return states[0]
    else:
        return np.random.choice(states, p = poss)

def get_reward(curr_state,action):
    return rewards[curr_state][action]

def random_act(state):
    act = get_actions(state)
    return np.random.choice(act)
        
def one_episode(startState):
    s_curr = startState
    sequence = ''
    tot_rewards = 0
    while is_end(s_curr) == False:
        act = random_act(s_curr)
        r = get_reward(s_curr,act)
        tot_rewards += r
        curr_s = 'state '+str(states[s_curr])+', action '+str(actions[act])+', reward '+str(tot_rewards)+'\n'
        sequence += curr_s
        s_next = get_next_state(s_curr,act)
        s_curr = s_next
    print(sequence + str(states[-1])+'\n')
    return tot_rewards

def policy_evaluation(policy):
    state_values = np.zeros(len(states))
    thresh = 0.001
    delta = 999
    iteration = 0
    while delta > thresh:
        iteration += 1
        delta = 0
        for s in range(len(states)-1):
            v_prev = state_values[s]
            action, probability = policy[s]
            v = 0
            for a, p in zip(action,probability):
                rewards = get_reward(s, a)
                s_next, pp = transition[s][a]
                for ns,n_p in zip(s_next,pp):
                    rewards += state_values[ns] * n_p
                v += rewards * p
            state_values[s] = v
            delta = max(delta,abs(state_values[s]-v_prev))
#         print('Iteration '+str(iteration)+ ', delta = '+str(delta))
    return state_values

def policy_iteration(initial_policy):
    policy = initial_policy
    stable = False
    iteration = 0
    while not stable:
        print('\nWhile iteration '+str(iteration))
        state_values = policy_evaluation(policy)
        print_state_values(state_values)
        
        # policy improvement lecture 14 page 16
        stable = True
        for s in range(len(states)-1):
            old_action = policy[s]
            possible_acts = get_actions(s)
            
            # maximize policy value
            best_action = None
            best_value = -999.0
            for a in possible_acts:
                v = get_reward(s, a)
                s_next, p_next = transition[s][a]
                for n_s,n_p in zip(s_next,p_next):
                    v += state_values[n_s] * n_p
                if v > best_value:
                    best_value = v
                    best_action = a
            policy[s] = ([best_action],[1])
            
            # stopping criterion
            if old_action != policy[s]:
                stable = False
        iteration += 1
        print_policy(policy)
    return policy

def print_policy(policy):
    string = ''
    for i in range(len(best_policy)):
        a,_ = best_policy[i]
        string += str(states[i]) + ': ' + actions[a[0]]+' | '
    print('Policy: '+string)
    return
    
def print_state_values(state_values):
    string = ''
    for i in range(len(state_values)):
        string  += str(states[i])+': '+str(state_values[i])+' | '
    print('State values: '+string)
    return

In [83]:
## 3a 1 and 3 The sequences of experience from each episode, including the Return observed in that episode. (pdf report)
tot_r = 0
for i in range(50):
    print('episode '+ str(i+1))
    r = one_episode(startState)
    tot_r += r
    
avg_r = tot_r/50
print('Average return for 50 episodes is {:0.3f}'.format(avg_r))

episode 1
state RU8p, action R, reward 0
state RU10p, action P, reward 2
state RU8a, action R, reward 2
state RU10a, action R, reward 2
11am class begins

episode 2
state RU8p, action P, reward 2
state TU10p, action P, reward 4
state RU10a, action S, reward 4
11am class begins

episode 3
state RU8p, action R, reward 0
state RU10p, action P, reward 2
state RU8a, action R, reward 2
state RU10a, action S, reward 2
11am class begins

episode 4
state RU8p, action P, reward 2
state TU10p, action P, reward 4
state RU10a, action S, reward 4
11am class begins

episode 5
state RU8p, action P, reward 2
state TU10p, action P, reward 4
state RU10a, action R, reward 4
11am class begins

episode 6
state RU8p, action S, reward -1
state RD10p, action R, reward -1
state RD8a, action R, reward -1
state RD10a, action S, reward 3
11am class begins

episode 7
state RU8p, action R, reward 0
state RU10p, action P, reward 2
state RU8a, action S, reward 1
state RD10a, action R, reward 5
11am class begins

episo

In [85]:
### 3a 2. The value of each state using Bellman equation
all_actions = [get_actions(s) for s in range(len(states) - 1)]
policies = [(a, [1/len(a) for _ in range(len(a))]) for a in all_actions]
state_values = policy_evaluation(policies)
for i in range(len(state_values)):
    print('The value of state {} is {:0.3f}'.format(states[i],state_values[i]))

Iteration 1, delta = 4.0
Iteration 2, delta = 3.5
Iteration 3, delta = 2.625
Iteration 4, delta = 1.5972222222222219
Iteration 5, delta = 0
The value of state RU8p is 3.514
The value of state TU10p is 1.667
The value of state RU10p is 2.500
The value of state RD10p is 5.375
The value of state RU8a is 1.333
The value of state RD8a is 4.500
The value of state TU10a is -1.000
The value of state RU10a is 0.000
The value of state RD10a is 4.000
The value of state TD10a is 3.000
The value of state 11am class begins is 0.000


In [89]:
### 3b policy iteration: learn the optimal policy
init_policy = [([0], [1]) for s in range(len(states) - 1)]
best_policy = policy_iteration(init_policy)


While iteration 0
State values: RU8p: 4.0 | TU10p: 2.0 | RU10p: 2.5 | RD10p: 6.5 | RU8a: 1.0 | RD8a: 5.0 | TU10a: -1.0 | RU10a: 0.0 | RD10a: 4.0 | TD10a: 3.0 | 11am class begins: 0.0 | 
Policy: RU8p: S | TU10p: R | RU10p: S | RD10p: P | RU8a: S | RD8a: P | TU10a: P | RU10a: P | RD10a: P | TD10a: P | 

While iteration 1
State values: RU8p: 5.5 | TU10p: 2.0 | RU10p: 4.0 | RD10p: 6.5 | RU8a: 3.0 | RD8a: 5.0 | TU10a: -1.0 | RU10a: 0.0 | RD10a: 4.0 | TD10a: 3.0 | 11am class begins: 0.0 | 
Policy: RU8p: S | TU10p: R | RU10p: S | RD10p: P | RU8a: S | RD8a: P | TU10a: P | RU10a: P | RD10a: P | TD10a: P | 

While iteration 2
State values: RU8p: 5.5 | TU10p: 3.0 | RU10p: 4.0 | RD10p: 6.5 | RU8a: 3.0 | RD8a: 5.0 | TU10a: -1.0 | RU10a: 0.0 | RD10a: 4.0 | TD10a: 3.0 | 11am class begins: 0.0 | 
Policy: RU8p: S | TU10p: R | RU10p: S | RD10p: P | RU8a: S | RD8a: P | TU10a: P | RU10a: P | RD10a: P | TD10a: P | 
